Tool for creating polygons for MGRS grid cells.

The purpose here is simple - given a point in space, determine its MGRS grid cell and then make a polygon of that cell

In [116]:
import mgrs
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
import folium

In [117]:
m = mgrs.MGRS()

In [212]:
latitude = 34.5
longitude = -104.0
mgrs_precision = np.arange(0,2)

In [213]:
for p in mgrs_precision:
    c = m.toMGRS(latitude,longitude,MGRSPrecision=p)
    base,easting,northing,precision = split_cell(c)
    print(base,easting,northing,precision)

13SEU None None 0
13SEU 9 1 1


- 1st component, 13, is the UTM grid zone. There are 60 of them  
- 2nd component, EU is the column (E) and row (U) letter of the 100,000 m^2 grid cell withn the UTM zone. Columns go from A-Z (omitting I and O) and rows go from A-V (omitting I and O)
- The confusing thing is that its not obvious what the letter code of adjacent cells is going to be, so we'll have to be able to deal with this

In [214]:
def split_cell(cellcode):
    
    decoded_cell = cellcode.decode()
    base_comp = decoded_cell[:5]
    precision = int((len(decoded_cell) - len(base_comp))/2)
    
    #Easting is distance in meters from the SW corner of the cell
    easting = decoded_cell[5:5+precision]
    #Northing is distance in meters from the base of the cell 
    northing = decoded_cell[5+precision:]
    
    if len(str(easting)) == 0:
        easting = None
        northing = None
        
        return base_comp, easting, northing, precision
    
    else:
        
        return base_comp, int(easting.strip()), int(northing.strip()), precision

In [284]:
def lon_lat_conversion(lon,lat,distlon,distlat):
    
    '''Get the coordinates of a new point lon + distlon, lat + distlat'''
    
    
    R = 6.731e6 #earth radius in meters
    distance_1_deg_lon = 2*np.pi*R*np.cos(lat*np.pi/180.0)/360.0
    distance_1_deg_lat = 2*np.pi*R/360.0
    
    new_lon = lon + distlon/distance_1_deg_lon
    new_lat = lat + distlat/distance_1_deg_lat
    
    return new_lon,new_lat


def precision_string(p):
    
    if p <= 1:
        return '%i%i'
    elif p == 2:
        return '%02i%02i'
    elif p == 3:
        return '%03i%03i'
    elif p == 4:
        return '%04i%04i'
    elif p == 5:
        return '%05i%05i'
    elif p == 6:
        return '%06i%06i'


def find_corners(cell):
    
    '''Return the coodinates of the four corners of this MGRS grid cell'''
    
    # . (lat_NW,lon_NW) -- .(lat_NE,lon_NE)
    # |                    |
    # |                    | 
    # |                    |
    # . (given) ---------- .(lat_SE,lon_SE)
    

    basecomp, easting, northing, precision = split_cell(cell)
    
    #Get the length of the cell side
    side_length = 1.1*10000/10**(precision-1)
    
    if precision > 0:
        
        #Get the coordinates of the SW corner of the cell
        pstring = precision_string(precision)
        cellID = basecomp+pstring %(easting,northing)
        latitude_SW,longitude_SW = m.toLatLon(cellID.encode())
        
        #initial attempt to find numerical neighbors (assuming we don't have to extend into the next cell)

        east_comp = easting + 1
        north_comp = northing + 1
    
        
        #Get the southeast cell coordinates
        try:
            c_southeast = basecomp+pstring %(east_comp,northing)
            latitude_SE, longitude_SE = m.toLatLon(c_southeast.encode())
        except:
            latitude_SE = None
            longitude_SE = None
        
        #Get the northeast cell coordinates
        try:
            c_northeast = basecomp+pstring %(east_comp,north_comp)
            latitude_NE, longitude_NE = m.toLatLon(c_northeast.encode())
        except:
            latitude_NE = None
            longitude_NE = None
        
        #Get the northwest cell coordinates
        try:
            c_northwest = basecomp+pstring %(easting,north_comp)
            latitude_NW, longitude_NW = m.toLatLon(c_northwest.encode())
        except:
            latitude_NW = None
            longitude_NW = None
            
        if len(str(east_comp)) > len(str(easting)):
            print('Need to check the ID of eastern adjacent 100,000 m^2 cell')
            
            nlon,nlat = lon_lat_conversion(longitude_SW,latitude_SW,distlon=side_length,distlat=0)
            
            #This is the cell directly east of our cell
            c = m.toMGRS(nlat,nlon,MGRSPrecision=precision)
            latitude_SE, longitude_SE = m.toLatLon(c)
            
            nlon,nlat = lon_lat_conversion(longitude_SW,latitude_SW,distlon=side_length,distlat=side_length)            
            #This is the cell northeast of our cell
            c = m.toMGRS(nlat,nlon,MGRSPrecision=precision)
            latitude_NE, longitude_NE = m.toLatLon(c)
                        
        if len(str(north_comp)) > len(str(northing)):
            print('Need to check the ID of northern adjacent 100,000 m^2 cell')
            
            nlon,nlat = lon_lat_conversion(longitude_SW,latitude_SW,distlon=0,distlat=side_length)
            
            #This is the cell directly north of our cell
            c = m.toMGRS(nlat,nlon,MGRSPrecision=precision)
            latitude_NW, longitude_NW = m.toLatLon(c)
            
            nlon,nlat = lon_lat_conversion(longitude_SW,latitude_SW,distlon=side_length,distlat=side_length)            
            #This is the cell northeast of our cell
            c = m.toMGRS(nlat,nlon,MGRSPrecision=precision)
            latitude_NE, longitude_NE = m.toLatLon(c)
            
        
    else:
        
        #Get the coordinates of the SW corner of the cell 
        cellID = basecomp
        latitude_SW,longitude_SW = m.toLatLon(cellID.encode())
       
        elon,slat = lon_lat_conversion(longitude_SW,latitude_SW,distlon=side_length,distlat=0)
        c = m.toMGRS(slat,elon,MGRSPrecision=precision)
        latitude_SE, longitude_SE = m.toLatLon(c)
        
        elon,nlat = lon_lat_conversion(longitude_SW,latitude_SW,distlon=side_length,distlat=side_length)
        c = m.toMGRS(nlat,elon,MGRSPrecision=precision)
        latitude_NE, longitude_NE = m.toLatLon(c)
        
        wlon,nlat = lon_lat_conversion(longitude_SW,latitude_SW,distlon=0,distlat=side_length)
        c = m.toMGRS(nlat,wlon,MGRSPrecision=precision)
        latitude_NW, longitude_NW = m.toLatLon(c)
    
    
    return {'southwest':(latitude_SW,longitude_SW),'southeast':(latitude_SE,longitude_SE),'northeast':(latitude_NE,longitude_NE),\
            'northwest':(latitude_NW,longitude_NW)}
        

In [285]:
c

b'13SEU91'

In [262]:
corner_coords = find_corners(c)

%i%i
13SEU91
Need to check the ID of eastern adjacent 100,000 m^2 cell
11000.0 0
34.42756368233047 -104.02050629105577
34.42756368233047 -103.90698817743495
11000.0 11000.0
34.42756368233047 -104.02050629105577
34.5211981459531 -103.90698817743495


In [243]:
corner_coords

{'southwest': (34.42756368233047, -104.02050629105577),
 'southeast': (34.42664369753953, -103.91169010432205),
 'northeast': (34.51680961373913, -103.91051885530943),
 'northwest': (34.51773268834001, -104.01945210276241)}

In [286]:
def make_mgrs_polgon(cc):
    
    lat_point_list = [cc['southwest'][0], cc['southeast'][0], cc['northeast'][0] ,cc['northwest'][0],cc['southwest'][0]]
    lon_point_list = [cc['southwest'][1], cc['southeast'][1], cc['northeast'][1] ,cc['northwest'][1],cc['southwest'][1]]
    polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
    return polygon_geom


    crs = {'init': 'epsg:4326'}
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])       
    return polygon

In [296]:
def make_grid_cells(precision=0,ncells=300):
    
    polys = [None]*ncells
    for i in range(ncells):
        
        randlat = np.random.uniform(-50,50,1)[0]
        randlon = np.random.uniform(-179,179,1)[0]
        c = m.toMGRS(randlat,randlon,MGRSPrecision=precision)
        cc = find_corners(c)
        
        polygon = make_mgrs_polgon(cc)
        polys[i] = polygon
    
    crs = {'init': 'epsg:4326'}
    polygon_data = gpd.GeoDataFrame(index=np.arange(len(polys)), crs=crs, geometry=polys) 
    return polygon_data        

In [297]:
polydata = make_grid_cells()

In [298]:
mapobj = folium.Map([polydata['geometry'].iloc[0].centroid.y, polydata['geometry'].iloc[0].centroid.x], zoom_start=6, tiles='cartodbpositron')
folium.GeoJson(polydata).add_to(mapobj)
folium.LatLngPopup().add_to(mapobj)
mapobj